In [1]:
import os
import random
import sys


## Package
import glob 
import IPython.display as ipd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as py
import plotly.tools as tls
import seaborn as sns
import scipy.io.wavfile
import tensorflow as tf
# import keras
# keras = tf.keras
py.init_notebook_mode(connected=True)


## Keras
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from tensorflow.keras.callbacks import  History, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from tensorflow.keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import utils
# NOTE: use utils.to_categorical
# from tensorflow.keras.utils import np_utils 
# from tensorflow.keras.utils import to_categorical


## Sklearn
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder


## Rest
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile
from tqdm import tqdm

input_duration=3

In [2]:
# load data
dir_list = os.listdir('ravdess-data/')
dir_list.sort()
print (dir_list)

['Actor_01', 'Actor_02', 'Actor_03', 'Actor_04', 'Actor_05', 'Actor_06', 'Actor_07', 'Actor_08', 'Actor_09', 'Actor_10', 'Actor_11', 'Actor_12', 'Actor_13', 'Actor_14', 'Actor_15', 'Actor_16', 'Actor_17', 'Actor_18', 'Actor_19', 'Actor_20', 'Actor_21', 'Actor_22', 'Actor_23', 'Actor_24']


In [3]:
# parse files
data_df = pd.DataFrame(columns=['path', 'source', 'actor', 'gender', 'intensity', 'statement', 'repetition', 'emotion'])

count = 0
for actor_folder in dir_list:
    file_list = os.listdir('ravdess-data/' + actor_folder)
    for f in file_list:
        nm = f.split('.')[0].split('-')
        path = 'ravdess-data/' + actor_folder + '/' + f
        src = int(nm[1])
        actor = int(nm[-1])
        emotion = int(nm[2])
        gender = 'female' if int(actor)%2 == 0 else 'male'
        intensity = 0 if nm[3] == '01' else 1
        statement = 0 if nm[4] == '01' else 1
        repeat = 0 if nm[5] == '01' else 1
        
        data_df.loc[count] = [path, src, actor, gender, intensity, statement, repeat, emotion]
        count+=1
        
        

In [4]:
print(len(data_df))
data_df.head()

1440


,path,source,actor,gender,intensity,statement,repetition,emotion
0,ravdess-data/Actor_01/03-01-01-01-01-01-01.wav,1,1,male,0,0,0,1
1,ravdess-data/Actor_01/03-01-01-01-01-02-01.wav,1,1,male,0,0,1,1
2,ravdess-data/Actor_01/03-01-01-01-02-01-01.wav,1,1,male,0,1,0,1
3,ravdess-data/Actor_01/03-01-01-01-02-02-01.wav,1,1,male,0,1,1,1
4,ravdess-data/Actor_01/03-01-02-01-01-01-01.wav,1,1,male,0,0,0,2


In [5]:
# finding a random file
filename = data_df.path[1021]
print(filename)

samples, sample_rate = librosa.load(filename)
sample_rate, samples

ravdess-data/Actor_18/03-01-01-01-01-02-18.wav


(22050, array([0., 0., 0., ..., 0., 0., 0.], dtype=float32))

In [6]:
len(samples), sample_rate

(77989, 22050)

In [ ]:
def log_specgram(audio, sample_rate, window_size=20, step_size=10, eps=1e-10):
    nperseg = int(round(window_size * sample_rate/1e3))
    noverlap = int(round(step_size * sample_rate/1e3))
    freqs, times, spec = signal.spectrogram(audio, fs=sample_rate, window='hann', nperseg=nperseg,noverlap=noverlap